<a href="https://colab.research.google.com/github/KyPython/RNA-3D-Folding/blob/main/Stanford_RNA_3D_Folding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Requirements & Installs/Imports***

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Stanford RNA 3D Folding

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Stanford RNA 3D Folding


In [ ]:
!nvidia-smi

Wed Apr  9 17:58:20 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P0             28W /   70W |     258MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Requirements File

In [ ]:
with open('requirements.txt', 'a') as f:
        f.write('\n')  # Add to requirements.txt'''

Load Cached Dependencies

In [ ]:
import sys
import os
#!mkdir -p /content/drive/MyDrive/colab_packages
# Target directory for installation
target_dir = '/content/drive/MyDrive/colab_packages'

# Ensure target directory exists
os.makedirs(target_dir, exist_ok=True)

# Install packages with --target
!pip install --no-cache-dir torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-2.6.0+cu124.html --target="{target_dir}"
sys.path.append(target_dir)
!pip install --upgrade cupy-cuda12x
!pip install --extra-index-url https://pypi.nvidia.com cudf-cu12 rmm
!pip install --no-cache-dir torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-2.6.0+cu124.html

Reinstall Dependencies (If Needed)

In [ ]:
import torch
print(f"Is CUDA available? {torch.cuda.is_available()}")
!pip install --upgrade pip
!pip install --target=/content/drive/MyDrive/colab_packages -r /content/drive/MyDrive/Stanford\ RNA\ 3D\ Folding/requirements.txt
!pip uninstall -y torch-geometric torch-scatter torch-sparse torch-cluster torch-spline-conv
!pip install torch-geometric

In [ ]:
import torch
print("CUDA Version:", torch.version.cuda)
print("Torch Version:", torch.__version__)
!nvcc --version

!pip install openmm
!pip install dask[distributed] cupy dask-cuda
import os
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import torch.nn as nn
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, global_mean_pool
from transformers import BertModel, BertTokenizer, EsmModel, EsmTokenizer
from torch.optim import Adam
import torch.optim as optim # Correct way to import optim if needed for general operations.
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from openmm import app, LangevinIntegrator, Platform
from openmm import unit as omm_unit
from openmm.openmm import *
from itertools import product
from IPython.display import display
from IPython import get_ipython
from IPython.display import display, HTML
import cudf
import cupy as cp #Make sure this is run in a cell before this code
import rmm

print("cuDF version:", cudf.__version__)
print("CuPy version:", cp.__version__) # Refer to cupy with the alias you gave in the import, cp.
print("RMM version:", rmm.__version__)
print("Torch Geometric Version:", torch_geometric.__version__)

# ***RNA Sequence Encoding***

RNA Sequence Encoding Methods Functions

In [ ]:
# Load RNA sequences from dataset files
def load_rna_sequences(file_path):
    df = pd.read_csv(file_path)
    return df['sequence'].dropna().tolist()  # Drop missing values and return as a list

# 1. One-Hot Encoding
def one_hot_encode(seq):
    mapping = {'A': [1, 0, 0, 0], 'U': [0, 1, 0, 0], 'G': [0, 0, 1, 0], 'C': [0, 0, 0, 1]}
    return np.array([mapping[base] for base in seq if base in mapping])

# 2. Integer Encoding
def integer_encode(seq):
    mapping = {'A': 0, 'U': 1, 'G': 2, 'C': 3}
    return [mapping[base] for base in seq if base in mapping]

# 3. k-mer Encoding (k=2)
def kmer_encode(seq, k=2):
    kmers = [''.join(kmer) for kmer in product('AUGC', repeat=k)]
    mapping = {kmer: i for i, kmer in enumerate(kmers)}
    return [mapping[seq[i:i+k]] for i in range(len(seq) - k + 1) if seq[i:i+k] in mapping]

# 4. Word Embedding using Torch (Random Initialization)
def get_word_embedding(seq, embedding_dim=4):
    vocab = {'A': 0, 'U': 1, 'G': 2, 'C': 3}
    indices = torch.tensor([vocab[base] for base in seq if base in vocab])
    embedding_layer = torch.nn.Embedding(num_embeddings=4, embedding_dim=embedding_dim)
    return embedding_layer(indices)

# 5. Positional Encoding (Transformer-style)
def positional_encoding(seq_length, d_model=4):
    """
    Computes positional encoding using sine and cosine functions.

    Args:
        seq_length (int): Length of the sequence.
        d_model (int): Dimension of the model.

    Returns:
        np.ndarray: Positional encoding matrix of shape (seq_length, d_model).
    """

    positions = np.arange(seq_length)[:, np.newaxis]
    div_term = np.exp(np.arange(0, d_model, 2) * -(np.log(10000.0) / d_model))
    pe = np.zeros((seq_length, d_model))
    pe[:, 0::2] = np.sin(positions * div_term)
    pe[:, 1::2] = np.cos(positions * div_term)
    return pe

RNA Sequence Encoding Execution

In [ ]:
# Paths to dataset files (Update these with actual file locations)
train_file = "/content/drive/MyDrive/Stanford RNA 3D Folding/Stanford_Data/train_sequences.csv"
validation_file = "/content/drive/MyDrive/Stanford RNA 3D Folding/Stanford_Data/validation_sequences.csv"
test_file = "/content/drive/MyDrive/Stanford RNA 3D Folding/Stanford_Data/test_sequences.csv"

# Load RNA sequences
train_sequences = load_rna_sequences(train_file)
validation_sequences = load_rna_sequences(validation_file)
test_sequences = load_rna_sequences(test_file)

# Select a sample RNA sequence for encoding (first one from train set)
rna_sequence = train_sequences[0] if train_sequences else "AUGC"

integer_encoded = integer_encode(rna_sequence)
print("\nInteger Encoding:", integer_encoded)

kmer_encoded = kmer_encode(rna_sequence, k=2)
print("\nk-mer Encoding:", kmer_encoded)

word_embedding = get_word_embedding(rna_sequence)
print("\nWord Embedding:\n", word_embedding)

# --- Execute the encoding methods after defining them ---

one_hot_encoded = one_hot_encode(rna_sequence)

integer_encoded = integer_encode(rna_sequence)
print("\nInteger Encoding:", integer_encoded)

kmer_encoded = kmer_encode(rna_sequence, k=2)
print("\nk-mer Encoding:", kmer_encoded)

word_embedding = get_word_embedding(rna_sequence)
print("\nWord Embedding:\n", word_embedding)

# Example usage:
pos_encoding = positional_encoding(len(rna_sequence))

# Print the full positional encoding
np.set_printoptions(threshold=np.inf)  # Print the full array
print("\nPositional Encoding:\n", pos_encoding)

# Convert the NumPy array to an HTML table for display
html_table = pd.DataFrame(pos_encoding).to_html(index=False)

# Display the HTML table in Jupyter Notebook
display(HTML(html_table))

# ***3D Structural Data***

3D Structural Data for Model Training Functions

In [ ]:
import cudf
import cupy as cp
import torch
import pandas as pd
import numpy as np

def load_3d_structure(file_path, n_samples=1000, chunk_size=5000):  # Reduced chunk size
    """Load 3D structure data using cuDF in chunks manually."""
    # Initialize an empty cuDF DataFrame to store the concatenated data
    df = cudf.DataFrame()

    # Read the CSV in chunks with Pandas, convert to cuDF, and concatenate
    for chunk in pd.read_csv(file_path, chunksize=chunk_size):
        df_chunk = cudf.DataFrame.from_pandas(chunk)
        df = cudf.concat([df, df_chunk], ignore_index=True)
        if len(df) > 100000:  # Optional: Stop once you have enough data
            break

    # Sample n_samples random rows from the DataFrame
    if n_samples < len(df):
        df = df.sample(n=n_samples, random_state=42)  # Set random_state for reproducibility

    # Extract relevant columns (residue ID & 3D coordinates)
    coords_columns = [col for col in df.columns if col.startswith(("x_", "y_", "z_"))]

    # Fill NaN values with 0 before converting to NumPy
    coords = df[coords_columns].fillna(0).to_numpy()

    return coords

# Normalize coordinates (mean-centered)
def normalize_coordinates(coords):
    return coords - np.mean(coords, axis=0)

# Compute pairwise distances using cuPy on GPU in smaller batches
def compute_pairwise_distances_gpu(coords, batch_size=500):  # Reduce batch size
    """Compute pairwise distances using cuPY on the GPU in smaller batches."""
    coords_gpu = cp.asarray(coords)  # Move data to GPU
    n = coords_gpu.shape[0]

    # Initialize an empty list to store the distance matrix chunks
    distance_matrix_chunks = []

    # Process data in batches
    for i in range(0, n, batch_size):
        batch_i = coords_gpu[i: min(i + batch_size, n)]

        # Reduce batch size further if needed
        if batch_i.shape[0] * coords_gpu.shape[0] > 1000000:  # Adjust based on memory availability
            smaller_batch_size = 1000000 // coords_gpu.shape[0]
            batch_i = batch_i[:smaller_batch_size]

        # Compute pairwise distances for the current batch
        distances_batch = cp.linalg.norm(batch_i[:, None, :] - coords_gpu[None, :, :], axis=2)
        distance_matrix_chunks.append(distances_batch)

    # Concatenate the chunks to get the full distance matrix
    distances_gpu = cp.concatenate(distance_matrix_chunks, axis=0)

    # Reduce precision to float16 if needed (optional for memory optimization)
    # distances_gpu = distances_gpu.astype(cp.float16)

    return cp.asnumpy(distances_gpu)  # Transfer results back to CPU

3D Structural Data Execution

In [ ]:
# Set the GPU visibility (use GPU with index 0)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Print versions
print("RMM version:", rmm.__version__)
print("cuDF version:", cudf.__version__)
print("cuPY version:", cp.__version__)

# File path for the 3D structure data
train_labels_file = "/content/drive/MyDrive/Stanford RNA 3D Folding/Stanford_Data/train_labels.csv"


# Load XYZ coordinates with reduced chunk size
xyz_coords = load_3d_structure(train_labels_file)
print("Raw XYZ Coordinates:\n", xyz_coords[:5])  # Show first 5 residues

normalized_coords = normalize_coordinates(xyz_coords)
print("\nNormalized Coordinates:\n", normalized_coords[:5])

# Compute pairwise distance matrix
distance_matrix = compute_pairwise_distances_gpu(normalized_coords)
print("\nPairwise Distance Matrix:\n", distance_matrix[:5, :5])

# Convert to PyTorch tensor for deep learning models
xyz_tensor = torch.tensor(normalized_coords, dtype=torch.float32)
distance_tensor = torch.tensor(distance_matrix, dtype=torch.float32)

print("\nXYZ Tensor Shape:", xyz_tensor.shape)
print("Distance Tensor Shape:", distance_tensor.shape)

RMM version: 25.02.00
cuDF version: 25.02.01
cuPY version: 13.4.1
Raw XYZ Coordinates:
 [[228.73199463 321.70901489 172.67500305]
 [301.00698853 128.70399475 220.96000671]
 [ 63.47200012  37.36800003  54.69800186]
 [-17.12899971 -55.125       -8.89900017]
 [ 13.08500004  -5.24599981 -45.72700119]]

Normalized Coordinates:
 [[ 186.76339866  272.71260091  107.9527409 ]
 [ 259.03839255   79.70758076  156.23774456]
 [  21.50340415  -11.62841396  -10.02426029]
 [ -59.09759568 -104.12141399  -73.62126232]
 [ -28.88359593  -54.2424138  -110.44926334]]

Pairwise Distance Matrix:
 [[  0.         211.67440592 349.39841346 485.20163021 448.44469377]
 [211.67440592   0.         303.98715253 433.40370253 414.68498005]
 [349.39841346 303.98715253   0.         138.18847599 120.1664845 ]
 [485.20163021 433.40370253 138.18847599   0.          68.97174856]
 [448.44469377 414.68498005 120.1664845   68.97174856   0.        ]]

XYZ Tensor Shape: torch.Size([1000, 3])
Distance Tensor Shape: torch.Size([1000

# ***Feature Engineering***

Feature Engineering Functions

In [ ]:
# Function to generate base-pairing map from dot-bracket notation
def dot_bracket_to_matrix(structure):
    stack = []
    matrix = np.zeros((len(structure), len(structure)), dtype=np.float32)

    for i, char in enumerate(structure):
        if char == '(':
            stack.append(i)
        elif char == ')':
            j = stack.pop()
            matrix[i, j] = matrix[j, i] = 1  # Symmetric pairing
    return matrix

# Function to compute nucleotide frequencies
def compute_nucleotide_frequencies(sequence):
    counts = {base: sequence.count(base) for base in 'AUGC'}
    total = len(sequence)
    return {base: count / total for base, count in counts.items()}

# Function to generate k-mer embeddings (k=2 for simplicity)
def kmer_encoding(sequence, k=2):
    kmers = [''.join(kmer) for kmer in product('AUGC', repeat=k)]
    mapping = {kmer: i for i, kmer in enumerate(kmers)}
    return [mapping[sequence[i:i+k]] for i in range(len(sequence) - k + 1)]

Feature Engineering Execution

In [ ]:
# Sample RNA secondary structure (Dot-Bracket Notation)
sample_sequence = "AUGCUAGC"
sample_structure = "..((..))"  # Example: Paired bases are indicated by ( )

# Generate base-pairing interaction matrix
pairing_matrix = dot_bracket_to_matrix(sample_structure)
print("Base-Pairing Interaction Matrix:\n", pairing_matrix)

nucleotide_freqs = compute_nucleotide_frequencies(sample_sequence)
print("\nNucleotide Frequencies:", nucleotide_freqs)

kmer_encoded = kmer_encoding(sample_sequence, k=2)
print("\nk-mer Encoding:", kmer_encoded)

# Convert base-pairing matrix to PyTorch tensor
pairing_tensor = torch.tensor(pairing_matrix, dtype=torch.float32)
print("\nPairing Tensor Shape:", pairing_tensor.shape)

Base-Pairing Interaction Matrix:
 [[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]]

Nucleotide Frequencies: {'A': 0.25, 'U': 0.25, 'G': 0.25, 'C': 0.25}

k-mer Encoding: [1, 6, 11, 13, 4, 2, 11]

Pairing Tensor Shape: torch.Size([8, 8])


# ***Structural Information***

Structural Information for Prediction Accuracy Functions

In [ ]:
# Function to generate base-pairing map from dot-bracket notation (GPU version)
def dot_bracket_to_matrix_gpu(structure):
    stack = []
    n = len(structure)
    matrix = cp.zeros((n, n), dtype=cp.float32)  # Create matrix on GPU

    for i, char in enumerate(structure):
        if char == '(':
            stack.append(i)
        elif char == ')':
            j = stack.pop()
            matrix[i, j] = matrix[j, i] = 1  # Symmetric pairing
    return matrix

# Function to compute nucleotide frequencies (GPU version)
def compute_nucleotide_frequencies_gpu(sequence):
    counts = {base: sequence.count(base) for base in 'AUGC'}
    total = len(sequence)
    freqs = {base: count / total for base, count in counts.items()}
    return freqs

# Function to generate k-mer embeddings (GPU version, k=2 for simplicity)
def kmer_encoding_gpu(sequence, k=2):
    kmers = [''.join(kmer) for kmer in product('AUGC', repeat=k)]
    mapping = {kmer: i for i, kmer in enumerate(kmers)}
    return [mapping[sequence[i:i+k]] for i in range(len(sequence) - k + 1)]

Structural Information Execution

In [ ]:
# Set the GPU visibility (use GPU with index 0)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Print versions
print("RMM version:", rmm.__version__)
print("cuDF version:", cudf.__version__)
print("cuPY version:", cp.__version__)

# Sample RNA secondary structure (Dot-Bracket Notation)
sample_sequence = "AUGCUAGC"
sample_structure = "..((..))"  # Example: Paired bases are indicated by ( )

# Generate base-pairing interaction matrix on GPU
pairing_matrix_gpu = dot_bracket_to_matrix_gpu(sample_structure)
print("Base-Pairing Interaction Matrix (GPU):\n", pairing_matrix_gpu)

nucleotide_freqs = compute_nucleotide_frequencies_gpu(sample_sequence)
print("\nNucleotide Frequencies:", nucleotide_freqs)

kmer_encoded = kmer_encoding_gpu(sample_sequence, k=2)
print("\nk-mer Encoding:", kmer_encoded)

# Convert base-pairing matrix to PyTorch tensor on GPU
pairing_tensor_gpu = torch.tensor(pairing_matrix_gpu.get(), dtype=torch.float32).cuda()  # Move to GPU
print("\nPairing Tensor Shape (GPU):", pairing_tensor_gpu.shape)


RMM version: 25.02.00
cuDF version: 25.02.01
cuPY version: 13.4.1
Base-Pairing Interaction Matrix (GPU):
 [[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]]

Nucleotide Frequencies: {'A': 0.25, 'U': 0.25, 'G': 0.25, 'C': 0.25}

k-mer Encoding: [1, 6, 11, 13, 4, 2, 11]

Pairing Tensor Shape (GPU): torch.Size([8, 8])


# ***Load & process RNA Data***

Load & Process RNA Data Functions

In [ ]:
import signal
import torch
from torch_geometric.data import Data
import numpy as np
import cupy as cp
from numba import cuda
from scipy.spatial import cKDTree

# Create a mapping using a dictionary (instead of a cupy.ndarray)
base_mapping_num = {
    ord('A'): 0,
    ord('C'): 1,
    ord('G'): 2,
    ord('T'): 3,
}

# Define base_mapping for CPU processing
base_mapping = {
    'A': 0,
    'C': 1,
    'G': 2,
    'T': 3,
    'N': 4,  # Handle unknown bases
}

class TimeoutException(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutException("Function execution timed out")

class SequenceGraph:
    """Handles sequence encoding and graph creation on GPU."""

    def __init__(self, sequence, coords):
        self.sequence = sequence
        self.coords = coords
        self.graph = self.create_graph()

    def create_graph(self):
        """Create a PyTorch Geometric graph on the GPU."""
        try:
            sequence_tensor = torch.tensor(self.sequence, dtype=torch.float32, device="cuda").unsqueeze(1)
            edge_index = torch.tensor([(i, i + 1) for i in range(len(self.sequence) - 1)], dtype=torch.long, device="cuda").t().contiguous()
            coords_tensor = torch.tensor(self.coords, dtype=torch.float32, device="cuda")

            return Data(x=sequence_tensor, edge_index=edge_index, pos=coords_tensor)
        except Exception as e:
            print(f"Graph creation failed: {e}")
            return None

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load RNA sequence and structure data using cuDF
rna_df = cudf.read_csv("/content/drive/MyDrive/Stanford RNA 3D Folding/Stanford_Data/train_sequences.csv")

# Convert the base_mapping dictionary into a CuPy array for the kernel
base_mapping_array = cp.array(list(base_mapping.values()))  # Using the values from base_mapping

def interpolate_missing_coordinates_gpu(coords):
    """Interpolates missing coordinates using KDTree on the GPU."""
    coords = cp.asarray(coords)
    coords_np = cp.asnumpy(coords)
    missing_indices = np.isnan(coords_np).any(axis=1)

    if np.all(missing_indices):
        print("Warning: All coordinates are missing. Imputing with mean values.")
        # Calculate mean of known coordinates across all sequences in your data
        mean_coords = np.nanmean(data[['x_1', 'y_1', 'z_1']].to_numpy(), axis=0)
        coords_np[:] = mean_coords  # Impute with mean
        return cp.asarray(coords_np)

    if not np.any(missing_indices):
        print("No missing coordinates found. Returning original data.")
        return coords

    known_indices = ~missing_indices
    known_coords = coords_np[known_indices]

    # Filter missing_coords to exclude rows with all NaNs
    missing_coords = coords_np[missing_indices]
    valid_missing_indices = ~np.isnan(missing_coords).all(axis=1)
    missing_coords = missing_coords[valid_missing_indices]

    if len(missing_coords) == 0:
        print("All missing coordinate rows contain only NaNs. Returning original data.")
        return coords

    tree = KDTree(known_coords)
    _, nearest_indices = tree.query(missing_coords)  # Find nearest neighbors

    for i, missing_index in enumerate(np.where(missing_indices)[0]):
        coords_np[missing_index] = known_coords[nearest_indices[i][0]]

    return cp.asarray(coords_np)


# Function to encode sequences using base_mapping
@cuda.jit
def integer_encode_kernel(seq_chars, base_mapping_num_array, result):
    idx = cuda.grid(1)
    if idx < len(seq_chars):  # Ensure we don't exceed array bounds
        base_char = seq_chars[idx]  # This should already be an integer
        # Manual checking whether the base_char is within the mapping
        if base_char == ord('A'):
            result[idx] = base_mapping_num_array[0]  # 'A'
        elif base_char == ord('C'):
            result[idx] = base_mapping_num_array[1]  # 'C'
        elif base_char == ord('G'):
            result[idx] = base_mapping_num_array[2]  # 'G'
        elif base_char == ord('T'):
            result[idx] = base_mapping_num_array[3]  # 'T'
        else:
            result[idx] = 4  # Handle unknown bases

# Function to encode sequences using base_mapping (GPU-accelerated with cuDF)
def encode_sequence_cpu(seq):
    """Encodes an RNA sequence using the base_mapping on the CPU."""
    # Use list comprehension and the get method for CPU-based encoding
    return [base_mapping.get(char, 4) for char in seq]

def contains_n(sequence):
    return 4 in sequence

# Class to manage sequence processing
class SequenceProcessor:
    def __init__(self, seq, coords):
        self.seq = seq
        self.coords = coords

    def process(self):
        """Encodes the sequence and interpolates missing coordinates."""
        encoded_seq = encode_sequence_cpu(self.seq)
        fixed_coords = interpolate_missing_coordinates_gpu(self.coords)
        return encoded_seq, fixed_coords

# Example Usage
if __name__ == "__main__":
    seq = "AGCTNAGT"
    coords = np.array([
        [1.0, 2.0, 3.0],
        [4.0, 5.0, 6.0],
        [np.nan, np.nan, np.nan],  # Missing coordinates
        [7.0, 8.0, 9.0]
    ])

    processor = SequenceProcessor(seq, coords)
    encoded_seq, fixed_coords = processor.process()

    print(f"Encoded sequence (GPU): {cp.asnumpy(encoded_seq)}")
    print(f"Interpolated coordinates (GPU): {cp.asnumpy(fixed_coords)}")

All missing coordinate rows contain only NaNs. Returning original data.
Encoded sequence (GPU): [0 2 1 3 4 0 2 3]
Interpolated coordinates (GPU): [[ 1.  2.  3.]
 [ 4.  5.  6.]
 [nan nan nan]
 [ 7.  8.  9.]]


Encoding Quick Test

In [ ]:
seq = "AGCTNAGT"
encoded_seq = encode_sequence_cpu(seq)  # Should work now!
print(f"Encoded sequence (CPU): {encoded_seq}")

Encoded sequence (CPU): [0, 2, 1, 3, 4, 0, 2, 3]


# ***Load & Process RNA Data Execution (Don't Run Again Unless You Want to Wait 10 Minutes)***

Create Encoded RNA Dataframe

In [ ]:
import cudf
import cupy as cp
import rmm
import pandas as pd
import numpy as np
import torch
from scipy.spatial import KDTree
from sklearn.preprocessing import LabelEncoder
from numba import cuda, njit

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load RNA sequence and structure data using cuDF
rna_df = cudf.read_csv("/content/drive/MyDrive/Stanford RNA 3D Folding/Stanford_Data/train_sequences.csv")
structure_df_cudf = cudf.read_csv(train_labels_file) # Load as cuDF DataFrame

# Clean 'ID' in structure_df_cudf to match base names (remove suffixes)
structure_df_cudf['ID'] = structure_df_cudf['ID'].str.split('_', expand=True)[0]

# Apply the encoding function to the 'sequence' column on CPU using `to_pandas` and `apply`
rna_df['encoded_sequence'] = rna_df['sequence'].to_pandas().apply(encode_sequence_cpu)

# Convert the 'encoded_sequence' column back to cudf Series if needed
rna_df['encoded_sequence'] = cudf.Series(rna_df['encoded_sequence'])

# Convert the 'encoded_sequence' column to a list of lists
encoded_sequences = rna_df['encoded_sequence'].to_arrow().to_pylist()

# Print the updated DataFrame with encoded sequences (optional)
print("Encoded RNA DataFrame (updated):")
print(rna_df.head())

# Load 3D structural data using pandas (if you need pandas for any specific operations)
# structure_df is already a pandas DataFrame, no need to convert again
structure_df = structure_df_cudf.to_pandas() # Convert to pandas when needed

# Clean both 'ID' in structure_df and 'target_id' in rna_df to match base names (remove suffixes)
structure_df['ID'] = structure_df['ID'].str.split('_', expand=True)[0]
rna_df['target_id'] = rna_df['target_id'].str.split('_', expand=True)[0]

# Ensure merge keys are of the same data type (e.g., string)
rna_df['target_id'] = rna_df['target_id'].astype(str)
structure_df['ID'] = structure_df['ID'].astype(str)

# Clean merge keys to ensure consistency (e.g., remove any extra characters or prefixes)
rna_df['target_id'] = rna_df['target_id'].str.replace('_.*', '', regex=True)
structure_df['ID'] = structure_df['ID'].str.replace('_.*', '', regex=True)

# ***GROUP BY and AGGREGATE before merging***
# Group by 'target_id' and take the first value for other columns to avoid duplicates
# Group and aggregate before merging, including 'encoded_sequence'
# Group and aggregate before merging, including 'encoded_sequence'
# Merge sequence and structure data, but keep 'encoded_sequence'
# CHANGED: Include 'encoded_sequence' in the merge
data = rna_df.merge(structure_df_cudf, left_on="target_id", right_on="ID", how='inner')

# Now you can group and aggregate
data = data.groupby('target_id', as_index=False).agg(
    {
        'sequence': 'first',
        'encoded_sequence': 'first',  # This will now be included
        'temporal_cutoff': 'first',
        'description': 'first',
        'all_sequences': 'first',
        # ... (other columns from structure_df_cudf)
        'resname': 'first',
        'resid': 'first',
        'x_1': 'first',
        'y_1': 'first',
        'z_1': 'first'
    }
)
structure_df_grouped = structure_df.groupby('ID').first().reset_index()

# Convert to pandas
# rna_df_grouped = rna_df_grouped.to_pandas() # Remove this conversion here
# structure_df_grouped is already a pandas DataFrame, no need to convert

# Merge sequence and structure data on cleaned and grouped IDs using pandas merge
# CHANGED: Convert structure_df_grouped to cudf before merge
structure_df_grouped_cudf = cudf.from_pandas(structure_df_grouped)

Encoded RNA DataFrame (updated):
  target_id                            sequence temporal_cutoff  \
0    1SCL_A       GGGUGCUCAGUACGAGAGGAACCGCACCC      1995-01-26   
1    1RNK_A  GGCGCAGUGGGCUAGCGCCACUCAAAAGGCCCAU      1995-02-27   
2    1RHT_A            GGGACUGACGAUCACGCAGUCUAU      1995-06-03   
3    1HLX_A                GGGAUAACUUCGGUUGUCCC      1995-09-15   
4    1HMH_E  GGCGACCCUGAUGAGGCCGAAAGGCCGAAACCGU      1995-12-07   

                                         description  \
0               THE SARCIN-RICIN LOOP, A MODULAR RNA   
1  THE STRUCTURE OF AN RNA PSEUDOKNOT THAT CAUSES...   
2  24-MER RNA HAIRPIN COAT PROTEIN BINDING SITE F...   
3  P1 HELIX NUCLEIC ACIDS (DNA/RNA) RIBONUCLEIC ACID   
4  THREE-DIMENSIONAL STRUCTURE OF A HAMMERHEAD RI...   

                                       all_sequences  \
0  >1SCL_1|Chain A|RNA SARCIN-RICIN LOOP|Rattus n...   
1  >1RNK_1|Chain A|RNA PSEUDOKNOT|null\nGGCGCAGUG...   
2  >1RHT_1|Chain A|RNA (5'-R(P*GP*GP*GP*AP*CP*UP*...   
3  

Merge Dataframe

In [ ]:
import dask.dataframe as dd
import torch
import numpy as np
from sklearn.neighbors import KDTree

rna_df = cudf.read_csv("/content/drive/MyDrive/Stanford RNA 3D Folding/Stanford_Data/train_sequences.csv")
train_labels_file = "/content/drive/MyDrive/Stanford RNA 3D Folding/Stanford_Data/train_labels.csv"
structure_df = cudf.read_csv(train_labels_file) # Load as cuDF DataFrame

# Convert rna_df and structure_df to Dask DataFrames
rna_df_dd = dd.from_pandas(rna_df, npartitions=4)  # Adjust number of partitions as needed
structure_df_dd = dd.from_pandas(structure_df, npartitions=4)

# Check columns in both dataframes before merge
# Call compute() on the Dask DataFrame itself to get the columns
print("RNA DataFrame columns before merge:", rna_df_dd.columns.to_list())
print("Structure DataFrame columns before merge:", structure_df_dd.columns.to_list())

# Merge sequence and structure data on cleaned and grouped IDs using Dask
data_dd = rna_df_dd.merge(structure_df_dd, left_on="target_id", right_on="ID", how="inner")

# Check columns in the merged dataframe
print("Merged DataFrame columns:", data_dd.columns.to_list())

# Convert Dask DataFrame to pandas for further processing
data = data_dd.compute()

print(f"Shape of merged data: {data.shape}")
print(f"First few rows of merged data:\n{data.head()}")

# Identify and print sequences with unknown characters ('N') using pandas
# Convert the 'encoded_sequence' column to a pandas Series for this operation
# Verify if 'encoded_sequence' column exists in 'data'
if 'encoded_sequence' in data.columns:
    # Column exists, proceed with processing
    unknown_sequences = data[data['encoded_sequence'].apply(lambda x: 4 in x)]
    print(f"Sequences with unknown characters: {unknown_sequences}")
else:
    # Column does not exist, handle the situation
    print("Error: 'encoded_sequence' column not found in the merged DataFrame.")

# Clean 'ID' in structure_df_cudf to match base names (remove suffixes)
structure_df_cudf['ID'] = structure_df_cudf['ID'].str.split('_', expand=True)[0]

# Apply the encoding function to the 'sequence' column on CPU using `to_pandas` and `apply`
rna_df['encoded_sequence'] = rna_df['sequence'].to_pandas().apply(encode_sequence_cpu)

# Convert the 'encoded_sequence' column back to cudf Series if needed
rna_df['encoded_sequence'] = cudf.Series(rna_df['encoded_sequence'])

# Convert into PyTorch tensors
encoded_sequences = rna_df['encoded_sequence'].to_arrow().to_pylist()
sequences = [torch.tensor(seq, dtype=torch.long).to(device) for seq in encoded_sequences]
coordinates = [torch.tensor(coords, dtype=torch.float32).to(device)
              for coords in data[['x_1', 'y_1', 'z_1']].values]

print(f"Sequences: {sequences[:5]}")  # Display first 5 sequences
print(f"Coordinates: {coordinates[:5]}")  # Display first 5 coordinates

# Check for missing coordinates using pandas
print(f"Check for missing coordinates in structure_df: {structure_df[['x_1', 'y_1', 'z_1']].isna().sum()}")

# Now, handle the data with missing coordinates
# Convert pandas DataFrame to NumPy array for KDTree
known_coordinates = data[['x_1', 'y_1', 'z_1']].dropna().to_numpy()

for index in data.index[data[['x_1', 'y_1', 'z_1']].isnull().any(axis=1)].to_pandas(): # Add .to_pandas() to convert to an iterable type
    coords = data.loc[index, ['x_1', 'y_1', 'z_1']].to_numpy()
    coords = coords.astype(np.float64)

    if not np.all(np.isnan(coords)):
        try:
            tree = KDTree(known_coordinates)
            missing_mask = np.isnan(coords)
            if missing_mask.any():
                distances, indices = tree.query(coords[missing_mask].reshape(-1, 3), k=1)
                coords[missing_mask] = known_coordinates[indices.flatten()]
                data.loc[index, ['x_1', 'y_1', 'z_1']] = coords  # Update DataFrame
        except ValueError:
            # Fallback to mean imputation if KDTree fails
            data.loc[index, ['x_1', 'y_1', 'z_1']] = np.nanmean(known_coordinates, axis=0)
    else:
        # Handle case where all values in coords are NaN
        data.loc[index, ['x_1', 'y_1', 'z_1']] = np.nanmean(known_coordinates, axis=0)

Create Sequence Graph

In [ ]:
import dask.dataframe as dd
import cupy as cp
import cudf
import dask_cudf
import torch
import numpy as np
from sklearn.neighbors import KDTree
from dask_cuda import LocalCUDACluster
from dask.distributed import Client

# Initialize Dask with GPU support
cluster = LocalCUDACluster()
client = Client(cluster)

# Load the RNA DataFrame with encoded sequences
rna_df = cudf.read_csv("/content/drive/MyDrive/Stanford RNA 3D Folding/Stanford_Data/train_sequences.csv")
rna_df['encoded_sequence'] = rna_df['sequence'].to_pandas().apply(encode_sequence_cpu)
rna_df['encoded_sequence'] = cudf.Series(rna_df['encoded_sequence']) # Convert back to cudf Series

# Load the structure DataFrame
train_labels_file = "/content/drive/MyDrive/Stanford RNA 3D Folding/Stanford_Data/train_labels.csv"
structure_df = cudf.read_csv(train_labels_file)

# Convert pandas DataFrames to Dask-cuDF DataFrames
# Use dask_cudf.from_delayed instead of dask_cudf.from_dask_dataframe
rna_ddf = dask_cudf.from_delayed(dd.from_pandas(rna_df[['target_id']], npartitions=4).to_delayed())  # Only include 'target_id'
structure_ddf = dask_cudf.from_delayed(dd.from_pandas(structure_df, npartitions=4).to_delayed())

# Merge the DataFrames on the appropriate columns (excluding 'encoded_sequence')
merged_ddf = rna_ddf.merge(structure_ddf, left_on="target_id", right_on="ID", how="inner")

# Convert Dask DataFrame to pandas for further processing
merged_df = merged_ddf.compute()

# Merge the encoded sequences back into the merged DataFrame
final_df = merged_df.merge(rna_df[['target_id', 'encoded_sequence']], on='target_id', how='left')


# Define a function to create graph data for a single row (without using SequenceGraph class)
def create_graph_data_for_row(row):
    """Create PyTorch Geometric graph data for a single row."""
    # ... (Your existing code to get encoded_seq and coords)

    # ***CHANGED: Get encoded_seq and coords from the row***
    encoded_seq = row['encoded_sequence']
    coords = row[['x_1', 'y_1', 'z_1']].values

    # Create the graph data using torch.tensor on the CPU
    sequence_tensor = torch.tensor(encoded_seq, dtype=torch.float32).unsqueeze(1)
    edge_index = torch.tensor([(i, i + 1) for i in range(len(encoded_seq) - 1)], dtype=torch.long).t().contiguous() if len(encoded_seq) > 1 else torch.empty((2, 0), dtype=torch.long)
    coords_tensor = torch.tensor(coords, dtype=torch.float32)

    # Create the Data object on the CPU
    graph_data = Data(x=sequence_tensor, edge_index=edge_index, pos=coords_tensor)

    # Move the Data object to the GPU if available
    if torch.cuda.is_available():
        graph_data = graph_data.to("cuda")

    return graph_data

# Apply the function to create graph data
graph_data = final_df.to_pandas().apply(create_graph_data_for_row, axis=1)
graph_data = graph_data.values.tolist() # Convert to a regular list

# Convert sequences and coordinates to PyTorch tensors
# Changed this to get encoded_sequence from the graph data
sequences = [torch.tensor(graph.x.cpu().numpy(), dtype=torch.long).to(device) for graph in graph_data]
coordinates = [torch.tensor(graph.pos.cpu().numpy(), dtype=torch.float32).to(device) for graph in graph_data]

# Check for missing coordinates
missing_coords = [i for i, coords in enumerate(coordinates) if coords.isnull().any()]

# Handle missing coordinates using KDTree
known_coordinates = data[['x_1', 'y_1', 'z_1']].dropna().to_numpy()

for index in data.index[data[['x_1', 'y_1', 'z_1']].isnull().any(axis=1)].to_pandas(): # Add .to_pandas() to convert to an iterable type
    coords = data.loc[index, ['x_1', 'y_1', 'z_1']].to_numpy()
    coords = coords.astype(np.float64)

    if not np.all(np.isnan(coords)):
        try:
            tree = KDTree(known_coordinates)
            missing_mask = np.isnan(coords)
            if missing_mask.any():
                distances, indices = tree.query(coords[missing_mask].reshape(-1, 3), k=1)
                coords[missing_mask] = known_coordinates[indices.flatten()]
                data.loc[index, ['x_1', 'y_1', 'z_1']] = coords  # Update DataFrame
        except ValueError:
            # Fallback to mean imputation if KDTree fails
            data.loc[index, ['x_1', 'y_1', 'z_1']] = np.nanmean(known_coordinates, axis=0)
    else:
        # Handle case where all values in coords are NaN
        data.loc[index, ['x_1', 'y_1', 'z_1']] = np.nanmean(known_coordinates, axis=0)

for i in missing_coords:
    coords = coordinates[i]
    if coords.isnull().any():
        distances, indices = tree.query(coords[coords.isnull()].reshape(-1, 3), k=1)
        coords[coords.isnull()] = known_coordinates[indices.flatten()]

# ***Models***

GNN For Spatial Interactions

In [ ]:
class GNNModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

Transformer for Sequence Depencencies (Lightweight Model)

In [ ]:
class TransformerModel(nn.Module):
    def __init__(self, vocab_size=4, hidden_dim=128):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.transformer = BertModel.from_pretrained("bert-base-uncased")
        self.fc = nn.Linear(768, hidden_dim)

    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(inputs_embeds=x).last_hidden_state.mean(dim=1)
        return self.fc(x)

Combine GNN & Transformer

In [ ]:
class HybridModel(nn.Module):
    def __init__(self, gnn_hidden=128, transformer_hidden=128, cnn_out=128, output_dim=3):
        super(HybridModel, self).__init__()
        self.gnn = GNNModel(input_dim=1, hidden_dim=gnn_hidden, output_dim=gnn_hidden)
        self.transformer = TransformerModel(vocab_size=4, hidden_dim=transformer_hidden)
        self.cnn3d = CNN3DModel(in_channels=1, out_dim=cnn_out)
        self.fc = nn.Linear(gnn_hidden + transformer_hidden + cnn_out, output_dim)

    def forward(self, graph, sequence, structure_3d):
        gnn_out = self.gnn(graph.x, graph.edge_index)
        gnn_out = global_mean_pool(gnn_out, batch=None)

        transformer_out = self.transformer(sequence)

        cnn_out = self.cnn3d(structure_3d)

        combined = torch.cat((gnn_out, transformer_out, cnn_out), dim=1)
        return self.fc(combined)

Training GNN

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. Train and Evaluate GNN
gnn_model = GNNModel(input_dim=1, hidden_dim=128, output_dim=3).to(device) # Output dim should be 3 for coordinates
gnn_optimizer = torch.optim.Adam(gnn_model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for i in range(len(graph_data)):
        graph, structure_3d = graph_data[i].to(device), coordinates[i].to(device)
        gnn_optimizer.zero_grad()
        predicted_coords = gnn_model(graph.x, graph.edge_index)  # GNN forward pass
        # Apply global_mean_pool to get a single prediction per graph
        predicted_coords = global_mean_pool(predicted_coords, batch=None)
        loss = rmsd_loss(predicted_coords, structure_3d)
        loss.backward()
        gnn_optimizer.step()
        total_loss += loss.item()
    print(f"GNN - Epoch {epoch+1}, RMSD Loss: {total_loss / len(graph_data)}")

print("Training complete!")
# Load the saved model
loaded_model = GNNModel().to(device)
loaded_model.load_state_dict(torch.load("/content/drive/MyDrive/Stanford RNA 3D Folding"))
loaded_model.eval()  # Set to evaluation mode

Train Transformers

In [ ]:
# 2. Train and Evaluate Transformer
transformer_model = TransformerModel(vocab_size=4, hidden_dim=128).to(device)
transformer_optimizer = torch.optim.Adam(transformer_model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for i in range(len(graph_data)):
        sequence, structure_3d = sequences[i].to(device), coordinates[i].to(device)
        transformer_optimizer.zero_grad()
        predicted_coords = transformer_model(sequence.unsqueeze(0)) # Transformer forward pass
        loss = rmsd_loss(predicted_coords, structure_3d)
        loss.backward()
        transformer_optimizer.step()
        total_loss += loss.item()
    print(f"Transformer - Epoch {epoch+1}, RMSD Loss: {total_loss / len(graph_data)}")

print("Training complete!")
# Load the saved model
loaded_model = TransformerModel().to(device)
loaded_model.load_state_dict(torch.load("/content/drive/MyDrive/Stanford RNA 3D Folding"))
loaded_model.eval()  # Set to evaluation mode

Hybrid GNN & Transformers Training

In [ ]:
model = HybridModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 50  # Adjust epochs as needed
best_loss = float('inf')  # Initialize best loss for saving the best model

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for i in range(len(graph_data)):
        graph, sequence, structure_3d = graph_data[i].to(device), sequences[i].to(device), coordinates[i].to(device)

        optimizer.zero_grad()

        # Forward pass
        predicted_coords = model(graph, sequence.unsqueeze(0), structure_3d.unsqueeze(0).unsqueeze(0)) # Add extra dimension for CNN3D

        # Compute RMSD loss
        loss = rmsd_loss(predicted_coords, structure_3d)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Print average loss per epoch
    avg_loss = total_loss / len(graph_data)
    print(f"Epoch {epoch+1}, RMSD Loss: {avg_loss}")

    # Save the best model
    if avg_loss < best_loss:
        best_loss = avg_loss
        # Create a directory for saving the model if it doesn't exist
        os.makedirs("saved_models", exist_ok=True)
        torch.save(model.state_dict(), "saved_models/best_hybrid_model.pth")
        print(f"Saved best model with loss {best_loss:.4f} to saved_models/best_hybrid_model.pth")

print("Training complete!")
# Load the saved model
loaded_model = HybridModel().to(device)
loaded_model.load_state_dict(torch.load("/content/drive/MyDrive/Stanford RNA 3D Folding"))
loaded_model.eval()  # Set to evaluation mode

RMSD (Roof Mean Square Deviation) Training Loss

In [ ]:
def rmsd_loss(predicted_coords, true_coords):
    """
    Compute the RMSD between predicted 3D coordinates and true coordinates.

    Args:
    predicted_coords (torch.Tensor): Predicted 3D coordinates (shape: [batch_size, 3])
    true_coords (torch.Tensor): True 3D coordinates (shape: [batch_size, 3])

    Returns:
    torch.Tensor: RMSD loss value
    """
    # Compute squared differences between predicted and true coordinates
    squared_diff = torch.square(predicted_coords - true_coords)

    # Compute RMSD
    rmsd = torch.sqrt(torch.mean(squared_diff))
    return rmsd

Evaluating Model Performance

In [ ]:
# 1. Compute RMSD (Root Mean Square Deviation)
def compute_rmsd(predicted_coords, true_coords):
    """
    Calculate RMSD between predicted and true 3D coordinates.
    """
    diff = predicted_coords - true_coords
    squared_diff = np.square(diff)
    mean_squared_diff = np.mean(squared_diff)
    rmsd = np.sqrt(mean_squared_diff)
    return rmsd

# 2. Compute TM-score (simplified version)
def compute_tm_score(predicted_coords, true_coords, d0=1.24 * (len(true_coords) ** (1/3)) - 1.8):
    """
    Compute a simplified TM-score.
    d0 is a scaling factor dependent on protein length (or RNA length in our case).
    """
    distances = np.linalg.norm(predicted_coords - true_coords, axis=1)
    tm_score = np.sum(1 / (1 + (distances / d0) ** 2)) / len(true_coords)
    return tm_score

# 3. Compute GDT-TS (Global Distance Test - Total Score)
def compute_gdt_ts(predicted_coords, true_coords, thresholds=[1, 2, 4, 8]):
    """
    Compute GDT-TS score, which measures the fraction of residues within certain distance thresholds.
    """
    distances = np.linalg.norm(predicted_coords - true_coords, axis=1)
    gdt_scores = [np.mean(distances < t) for t in thresholds]
    gdt_ts = np.mean(gdt_scores)
    return gdt_ts

# Sample Test Data
true_coords = np.array([[0, 0, 0], [1, 1, 1], [2, 2, 2], [3, 3, 3]])  # True 3D positions
predicted_coords = np.array([[0, 0.1, 0], [1.1, 1, 1], [2, 2.2, 2], [3, 3, 3.1]])  # Predicted positions

# Evaluate Model Performance
rmsd_value = compute_rmsd(predicted_coords, true_coords)
tm_score_value = compute_tm_score(predicted_coords, true_coords)
gdt_ts_value = compute_gdt_ts(predicted_coords, true_coords)

# Print results
print(f"RMSD: {rmsd_value:.4f}")
print(f"TM-score: {tm_score_value:.4f}")
print(f"GDT-TS: {gdt_ts_value:.4f}")

NameError: name 'true_coords' is not defined

Adam Training Optimizer

In [ ]:
# Set up optimizer with weight decay (L2 regularization)
learning_rate = 0.001
weight_decay = 1e-5  # Regularization strength (adjust if needed)
optimizer = Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# Set up learning rate scheduler (decay every 10 epochs)
scheduler = StepLR(optimizer, step_size=10, gamma=0.5)  # Halve learning rate every 10 epochs

# Training loop
epochs = 50  # Adjust based on your dataset size
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for i in range(len(graph_data)):
        graph, sequence, structure_3d = graph_data[i].to(device), sequences[i].to(device), coordinates[i].to(device)

        optimizer.zero_grad()

        # Forward pass
        predicted_coords = model(graph, sequence.unsqueeze(0), structure_3d.unsqueeze(0))

        # Compute RMSD loss
        loss = rmsd_loss(predicted_coords, structure_3d.unsqueeze(0))

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Step scheduler
    scheduler.step()

    # Print average loss per epoch
    print(f"Epoch {epoch+1}, RMSD Loss: {total_loss / len(graph_data)}")

    # Optionally, print learning rate at each epoch
    print(f"Learning Rate at Epoch {epoch+1}: {scheduler.get_last_lr()[0]}")

RNA/LSTM Model (Long Short Term Memory)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random

# Define RNA bases
RNA_BASES = ['A', 'U', 'G', 'C']

# -------------------- DATA AUGMENTATION FUNCTIONS --------------------

def mutate_rna(sequence, mutation_rate=0.1):
    """ Introduce random mutations while keeping valid RNA bases. """
    return ''.join([
        base if random.random() > mutation_rate else random.choice(RNA_BASES)
        for base in sequence
    ])

def shuffle_rna(sequence, window_size=5):
    """ Shuffle small segments to introduce sequence variability. """
    seq_list = list(sequence)
    for i in range(0, len(seq_list) - window_size, window_size):
        random.shuffle(seq_list[i:i+window_size])
    return ''.join(seq_list)

def perturb_3d_coordinates(coords, noise_level=0.5):
    """ Add random noise to 3D atomic coordinates. """
    noise = np.random.normal(scale=noise_level, size=coords.shape)
    return coords + noise

# -------------------- ONE-HOT ENCODING --------------------

def one_hot_encode(seq):
    """ Convert RNA sequence into a one-hot encoded matrix. """
    mapping = {'A': [1, 0, 0, 0], 'U': [0, 1, 0, 0], 'G': [0, 0, 1, 0], 'C': [0, 0, 0, 1]}
    return np.array([mapping[base] for base in seq])

# -------------------- LSTM MODEL --------------------

from Bio.PDB import PDBIO, Structure, Model, Chain, Residue, Atom

def save_pdb(pred_coords, output_file="predicted_structure.pdb"):
    """ Save predicted 3D coordinates as a PDB file. """
    structure = Structure.Structure("RNA")
    model = Model.Model(0)
    chain = Chain.Chain("A")

    for i, (x, y, z) in enumerate(pred_coords.detach().numpy()):
        residue = Residue.Residue((" ", i, " "), "A", " ")
        atom = Atom.Atom("P", (x, y, z), 1.0, 1.0, " ", "P", i, "P")
        residue.add(atom)
        chain.add(residue)

    model.add(chain)
    structure.add(model)

    io = PDBIO()
    io.set_structure(structure)
    io.save(output_file)

# Call this function after model inference
save_pdb(predicted_coords, "rna_prediction.pdb")

class RNASequenceModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNASequenceModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)  # Output predicts secondary structure

    def forward(self, x):
        lstm_out, _ = self.lstm(x)  # x: (batch_size, seq_length, input_size)
        out = self.fc(lstm_out[:, -1, :])  # Use last time step for prediction
        return out

# -------------------- MODEL INITIALIZATION --------------------

input_size = 4  # One-hot encoding for A, U, G, C
hidden_size = 128
output_size = 1  # Predicts secondary structure (e.g., binary classification)

model = RNASequenceModel(input_size=input_size, hidden_size=hidden_size, output_size=output_size)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()  # Mean Squared Error Loss for regression tasks

# -------------------- TRAINING DATA --------------------

rna_sequence = "AUGCGAUCGUAGC"  # Example sequence
target_structure = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]  # Example base-pairing binary labels

# Apply data augmentation
augmented_seq = mutate_rna(rna_sequence, mutation_rate=0.2)
augmented_seq = shuffle_rna(augmented_seq, window_size=4)

# Encode the RNA sequence
encoded_seq = one_hot_encode(augmented_seq)
encoded_seq = torch.tensor(encoded_seq, dtype=torch.float32).unsqueeze(0)  # Shape: (1, seq_length, input_size)
target_structure = torch.tensor(target_structure, dtype=torch.float32).unsqueeze(0)  # Shape: (1, seq_length)

# -------------------- TRAINING LOOP --------------------

epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    output = model(encoded_seq)

    # Compute loss
    loss = loss_fn(output, target_structure)

    # Backpropagation
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}')

Model Fine Tuning

In [ ]:
# -------------------- 1. DATA AUGMENTATION --------------------

def mutate_sequence(seq, mutation_rate=0.1):
    """ Randomly mutate the RNA sequence to introduce variability. """
    bases = ['A', 'U', 'G', 'C']
    seq_list = list(seq)
    for i in range(len(seq_list)):
        if np.random.rand() < mutation_rate:
            seq_list[i] = np.random.choice(bases)
    return "".join(seq_list)

# -------------------- 2. ONE-HOT ENCODING --------------------

def one_hot_encode(seq):
    """ Convert RNA sequence into one-hot encoded format. """
    mapping = {'A': [1, 0, 0, 0], 'U': [0, 1, 0, 0], 'G': [0, 0, 1, 0], 'C': [0, 0, 0, 1]}
    return np.array([mapping[base] for base in seq])

# -------------------- 3. FEATURE EXTRACTION (TRANSFER LEARNING) --------------------

esm_model = EsmModel.from_pretrained("facebook/esm2_t6_8M_UR50D")
esm_tokenizer = EsmTokenizer.from_pretrained("facebook/esm2_t6_8M_UR50D")

def extract_features(sequence):
    """ Extract RNA sequence embeddings using ESM-2 (transfer learning). """
    inputs = esm_tokenizer(sequence, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = esm_model(**inputs)
    return outputs.last_hidden_state.squeeze(0)  # Shape: (seq_length, embedding_dim)

# -------------------- 4. DATASET CLASS --------------------

class RNA3DDataset(Dataset):
    def __init__(self, rna_sequence, secondary_structure, atom_coordinates):
        self.rna_sequence = rna_sequence
        self.secondary_structure = secondary_structure
        self.atom_coordinates = atom_coordinates

    def __len__(self):
        return len(self.rna_sequence)

    def __getitem__(self, idx):
        mutated_seq = mutate_sequence(self.rna_sequence)  # Apply data augmentation
        seq_features = extract_features(mutated_seq)  # Use transfer learning
        sec_structure = self.secondary_structure[idx]
        coords = self.atom_coordinates[idx]
        return seq_features, torch.tensor(sec_structure, dtype=torch.float32), torch.tensor(coords, dtype=torch.float32)

# -------------------- 5. MODEL: TRANSFORMER + LSTM --------------------

class RNA3DModel(nn.Module):
    def __init__(self, input_size=320, hidden_size=128, output_size=3):
        super(RNA3DModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, 64)
        self.fc2 = nn.Linear(64, output_size)  # Predicts (X, Y, Z) coordinates

    def forward(self, seq_input, sec_input):
        lstm_out, _ = self.lstm(seq_input)
        x = self.fc1(lstm_out[:, -1, :])
        x = torch.relu(x)
        coords = self.fc2(x)
        return coords

# -------------------- 6. LOSS FUNCTION --------------------

def rmsd_loss(pred_coords, true_coords):
    """ Compute RMSD loss for 3D coordinates. """
    return torch.sqrt(torch.mean((pred_coords - true_coords) ** 2))

# -------------------- 7. TRAINING --------------------

# Sample Data (for demo)
rna_sequence = "AUGCGAUCGUAGC"
secondary_structure = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]
atom_coordinates = np.random.rand(len(rna_sequence), 3)  # Simulated 3D coordinates

# DataLoader
dataset = RNA3DDataset(rna_sequence, secondary_structure, atom_coordinates)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

# Model, Optimizer
model = RNA3DModel()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    for seq_input, sec_input, true_coords in dataloader:
        optimizer.zero_grad()
        pred_coords = model(seq_input, sec_input)
        loss = rmsd_loss(pred_coords, true_coords)
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# -------------------- 8. TESTING --------------------

sample_features = extract_features(rna_sequence).unsqueeze(0)
predicted_coords = model(sample_features, torch.tensor([1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1], dtype=torch.float32).unsqueeze(0))

print("Predicted 3D coordinates:", predicted_coords)

Post Prediction Energy Minimization

In [ ]:
def energy_minimize_pdb(pdb_file, output_file="minimized.pdb"):
    """
    Perform energy minimization on an RNA structure using OpenMM.

    Args:
    - pdb_file (str): Path to the input PDB file.
    - output_file (str): Path to save the minimized structure.

    Returns:
    - Minimized structure saved in PDB format.
    """
    # Load RNA structure from PDB
    pdb = app.PDBFile(pdb_file)
    forcefield = app.ForceField("amber99sb.xml", "amber99_obc.xml")

    # Create OpenMM system
    system = forcefield.createSystem(
        pdb.topology, nonbondedMethod=app.NoCutoff, constraints=app.HBonds
    )

    # Define integrator for energy minimization
    integrator = LangevinIntegrator(300 * omm_unit.kelvin, 1 / omm_unit.picosecond, 0.002 * omm_unit.picoseconds)

    # Use CPU or GPU for simulation
    platform = Platform.getPlatformByName("CPU")  # Change to "CUDA" for GPU

    # Set up simulation
    simulation = app.Simulation(pdb.topology, system, integrator, platform)
    simulation.context.setPositions(pdb.positions)

    # Energy minimization
    print("Minimizing energy...")
    simulation.minimizeEnergy()

    # Save minimized structure
    positions = simulation.context.getState(getPositions=True).getPositions()
    with open(output_file, "w") as f:
        app.PDBFile.writeFile(pdb.topology, positions, f)

    print(f"Minimized structure saved to {output_file}")

# Example: Run energy minimization on an RNA structure
pdb_file = "predicted_rna.pdb"  # Replace with actual predicted structure
energy_minimize_pdb(pdb_file)

FileNotFoundError: [Errno 2] No such file or directory: 'predicted_rna.pdb'

Fine Tuned Model With Simulation Learning

In [ ]:
class CorrectionModel(nn.Module):
    def __init__(self, input_dim=3, hidden_dim=64, output_dim=3):
        super(CorrectionModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

correction_model = CorrectionModel()
optimizer = optim.Adam(correction_model.parameters(), lr=0.001)